# Assignment 1:

In this assignment, you will re-implement the core functions in `my.py` for relational schema reasoning: equality, FD violations, closure, BCNF decomposition, projection, join, and product. Use the notebook cells below to test your code.

In [1]:
# [THIS IS READ-ONLY]
import my

In [2]:
# [THIS IS READ-ONLY]
from utils import fd_str, schema_str
from importlib import reload
reload(my);

## Getting to know `REL` class.

Take a look at the `REL` class in `my.py`.  Some functions are already given.  The `REL` class will hold the schema information (attributes) and functional dependencies, as well as the tuples.

In [3]:
# [THIS IS READ-ONLY]
R = my.REL(
    ["a", "b", "c"],
    rows = [
        (1,2,3),
        (1,4,5),
        (1,2,3)
    ])

R.df()

No FDs.


,a,b,c
0,1,2,3
1,1,4,5


## Equality

Implement the `REL.__eq__` method.  See `my.py`.

In [ ]:
# [THIS IS READ-ONLY]
# @check
# @title: equality

R1 = my.REL(["a", "b", "c"],
            rows = [
                ("a1", "b1", "c1"),
                ("a2", "b2", "c2"),
            ])

R2 = my.REL(["b", "a", "c"],
            rows = [
                ("b1", "a1", "c1"),
                ("b2", "a2", "c2"),
            ])

R3 = my.REL(["a", "b", "c"],
            rows = [
                ("a2", "b2", "c2"),
                ("a1", "b1", "c1"),
            ])

R4 = my.REL(["a", "b", "c"],
            rows = [
                ("a1", "b1", "x"),
                ("a2", "b2", "c2"),
            ])

assert R1 == R2
assert R1 == R3
assert R2 == R3

assert R1 != R4
assert R2 != R4
assert R3 != R4

## FD Satisfaction

Implement `REL.satisfies` that checks if the current tuples in the relation satisfies a given FD.

In [ ]:
# [THIS IS READ-ONLY]
# @check
# @title: FD satisfaction

R = my.REL(
    ["a", "b", "c"],
    rows = [
        (1,2,3),
        (1,4,5),
    ])

fd = ({"a"}, {"c"})
print(f"SAT: {fd_str(fd)}?", R.satisfies(fd))

fd = ({"b"}, {"c"})
print(f"SAT: {fd_str(fd)}?", R.satisfies(fd))

## FD addition

Implement the `REL.add_fd` method which allows addition of FDs into the REL schema.

In [ ]:
# [THIS IS READ-ONLY]
# @check
# @title: adding FDs

R = my.REL(
    ["a", "b", "c"],
    rows = [
        (1,2,3),
        (1,4,5),
    ])

fds = [
    ({"a"}, {"c"}),
    ({"a", "c"}, {"b"}),
]
for fd in fds:
    try:
        R.add_fd(fd)
        print(f"{fd_str(fd)} is accepted.")
    except:
        print(f"{fd_str(fd)} is rejected.")

print(R.df())

## Data loading

Implement the `load_csv(...)` function in `my.py`.  This loads a relation from a CSV file.

In [ ]:
# [THIS IS READ-ONLY]
# @check
# @title: load from csv

S = my.load_csv('data/r1.csv')
T = my.load_csv('data/r2.csv')

print(S.df())
print(T.df())

## Projection

Implement the `project(...)` relational operator in `my.py`.

In [ ]:
# [THIS IS READ-ONLY]
# @check
# @title: projection

my.project(T, ["id2", "name"]).df()

## Natural Join

Implement the natural join operator in `my.py`.

In [ ]:
# [THIS IS READ-ONLY]
# @check
# @title: natural join

my.natural_join(S, T).df()

## Closure set under FDs

Implement the `REL.closure` method that computes the closure of a given attribute set using the FDs in the REL schema.

In [ ]:
# [THIS IS READ-ONLY]
# @check
# @title: closure

R = my.REL(["a", "b", "c", "d", "e"])
fds = [
    ({"a"}, {"b", "c"}),
    ({"c"}, {"b"}),
    ({"b"}, {"d"}),
    ({"a", "e"}, {"d"}),
]
for fd in fds:
    R.add_fd(fd)

print("{a}+ = ", sorted(R.closure({"a"})))
print("{b}+ = ", sorted(R.closure({"b"})))
print("{ae}+ = ", sorted(R.closure({"a", "e"})))


## BCNF violations

Implement the `REL.bcnf_violations` method that finds all FDs in the schema that violates the BCNF condition.

In [ ]:
# [THIS IS READ-ONLY]
# @check
# @title: BCNF violations

for fd in sorted(R.bcnf_violations(), key=lambda fd: tuple(fd[0])):
    print(f"{fd[0]} -> {fd[1]} violates BCNF")

## A realistic dataset

Let's load a more realistic dataset that describes projects, members and skills of members.

The functional dependencies are specified.

In [ ]:
# [THIS IS READ-ONLY]
# @check
# @title: a real example

PROJ = my.load_csv('data/data.csv')
fds = [
    ({'project_title'}, {'deadline', 'lead', 'client'}),
    ({'member_name'}, {'age', 'email', 'city'}),
    ({'email'}, {'member_name'}),
]
for fd in fds:
    PROJ.add_fd(fd)
PROJ.df()

## Checking FDs against the data

We should not be surprised that `project_title` cannot determine a unique `skill` value.

In [ ]:
# [THIS IS READ-ONLY]
# @check
# @title: FD violations


print(
    "SAT project_title -> skill?",
    PROJ.satisfies(({"project_title"}, {"skill"}))
)


## A candidiate key

We can check that a candidate key is `{project_title, email, skill}` because its closure is the whole relation attribute set.

In [ ]:
# [THIS IS READ-ONLY]
# @check
# @title: project_title closure

sorted(PROJ.closure({'project_title', 'email', 'skill'}))

## BCNF decomposition

Implement the `bcnf_decompose(...)` function in `my.py`.  It will apply iterative projection to construct a series of relations such that:

- Each relation satisfies the BCNF condition.
- The original relation can be reconstructed by natural joins.

In [ ]:
# [THIS IS READ-ONLY]
# @check
# @title: BCNF decomposition
# @grade: 3

Rs = my.bcnf_decompose(PROJ)
Rs.sort(key=schema_str)

for R in Rs:
    print(schema_str(R))
    print()

In [ ]:
# [THIS IS READ-ONLY]
# @check
# @title: satisfies BCNF
# @grade: 3

for R in Rs:
    print(R.bcnf_violations())

In [ ]:
# [THIS IS READ-ONLY]
# @check
# @title: lossless join
# @grade: 3

lossjoin = None
for R in Rs:
    lossjoin = my.natural_join(lossjoin, R) if lossjoin else R

print(
    "Lossless join?", lossjoin == PROJ)